# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-pre-processed-FASTA-files" data-toc-modified-id="Generate-pre-processed-FASTA-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate pre-processed FASTA files</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-low-spike" data-toc-modified-id="Prepare-low-spike-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare low-spike</a></div><div class="lev2 toc-item"><a href="#Prepare-high-spikes" data-toc-modified-id="Prepare-high-spikes-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Prepare high-spikes</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulate MS-data</a></div><div class="lev1 toc-item"><a href="#Post-process-data" data-toc-modified-id="Post-process-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Post-process data</a></div><div class="lev1 toc-item"><a href="#Normalization" data-toc-modified-id="Normalization-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Normalization</a></div>

# Generate pre-processed FASTA files

Using the custom-made `seqtk generate_spikein.py` script.

## Setup

In [1]:
run=batch2
mkdir ${run}

In [2]:
sample_names_low="l1 l2 l3"
sample_names_high="h1 h2 h3"
samples="${sample_names_low} ${sample_names_high}"

In [3]:
echo -e "name\tbiorepgroup\ttechrepgroup\tcondition" > ${run}/design.tsv
echo -e "l1\t1\t1\tlow" >> ${run}/design.tsv
echo -e "l2\t2\t1\tlow" >> ${run}/design.tsv
echo -e "l3\t3\t1\tlow" >> ${run}/design.tsv
echo -e "h1\t1\t1\thigh" >> ${run}/design.tsv
echo -e "h2\t2\t1\thigh" >> ${run}/design.tsv
echo -e "h3\t3\t1\thigh" >> ${run}/design.tsv

cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition
l1	1	1	low
l2	2	1	low
l3	3	1	low
h1	1	1	high
h2	2	1	high
h3	3	1	high


## Prepare low-spike

In [4]:
for name in ${sample_names_low}; do
    echo "Generating sample: ${name}"
    lfqtk generate_spikein \
        --background_fa data/uniprot_ecoli.fasta \
        --spikein_fa data/uniprot_potato.fasta \
        --output_fa ${run}/${name}.fa \
        --offset_mean 0 \
        --offset_std 0 \
        --back_int 1000000 \
        --back_noise_std 50000 \
        --back_count 100 \
        --spike_int 1000000 \
        --spike_noise_std 50000 \
        --spike_count 20 \
        --verbose
done

Generating sample: l1
5268 entries loaded from data/uniprot_ecoli.fasta as background
858 entries loaded from data/uniprot_potato.fasta as spike-in
100 entries picked as background, 20 as spike-in
120 entries written to batch2/l1.fa
Generating sample: l2
5268 entries loaded from data/uniprot_ecoli.fasta as background
858 entries loaded from data/uniprot_potato.fasta as spike-in
100 entries picked as background, 20 as spike-in
120 entries written to batch2/l2.fa
Generating sample: l3
5268 entries loaded from data/uniprot_ecoli.fasta as background
858 entries loaded from data/uniprot_potato.fasta as spike-in
100 entries picked as background, 20 as spike-in
120 entries written to batch2/l3.fa


## Prepare high-spikes

In [5]:
for name in ${sample_names_high}; do
    echo "Generating sample: ${name}"
    lfqtk generate_spikein \
        --background_fa data/uniprot_ecoli.fasta \
        --spikein_fa data/uniprot_potato.fasta \
        --output_fa ${run}/${name}.fa \
        --offset_mean 0 \
        --offset_std 0 \
        --back_int 1000000 \
        --back_noise_std 50000 \
        --back_count 100 \
        --spike_int 2000000 \
        --spike_noise_std 50000 \
        --spike_count 20 \
        --verbose
done

Generating sample: h1
5268 entries loaded from data/uniprot_ecoli.fasta as background
858 entries loaded from data/uniprot_potato.fasta as spike-in
100 entries picked as background, 20 as spike-in
120 entries written to batch2/h1.fa
Generating sample: h2
5268 entries loaded from data/uniprot_ecoli.fasta as background
858 entries loaded from data/uniprot_potato.fasta as spike-in
100 entries picked as background, 20 as spike-in
120 entries written to batch2/h2.fa
Generating sample: h3
5268 entries loaded from data/uniprot_ecoli.fasta as background
858 entries loaded from data/uniprot_potato.fasta as spike-in
100 entries picked as background, 20 as spike-in
120 entries written to batch2/h3.fa


# Simulate MS-data

Generate the OpenMS tool `MSSimulator`.

In [7]:
for sample in ${samples}; do
    echo "Processing sample: ${sample}"
    MSSimulator \
        -in ${run}/${sample}.fa \
        -out_fm ${run}/${sample}.featureXML \
        -out_id ${run}/${sample}.idXML \
        > ${run}/${sample}.mssim.log
done


l1
l2
l3
h1
h2
h3


In [10]:
for sample in ${samples}; do \
    echo "Processing sample: ${sample}"
    IDMapper \
        -id ${run}/${sample}.idXML \
        -in ${run}/${sample}.featureXML \
        -out ${run}/${sample}.mapped.featureXML \
        > ${run}/${sample}.mapped.featureXML.log
done

Processing sample: l1
Processing sample: l2
Processing sample: l3
Processing sample: h1
Processing sample: h2
Processing sample: h3


In [12]:
for sample in ${samples}; do 
    echo "Aligning sample: ${sample}"
    MapAlignerPoseClustering \
        -in ${run}/${sample}.mapped.featureXML \
        -out ${run}/${sample}.mapped.aligned.featureXML \
        > ${run}/${sample}.mapped.aligned.featureXML.log
done

Aligning sample: l1
Aligning sample: l2
Aligning sample: l3
Aligning sample: h1
Aligning sample: h2
Aligning sample: h3


In [15]:
FeatureLinkerUnlabeledQT \
    -in ${run}/*.mapped.aligned.featureXML \
    -out ${run}/combined.consensusXML

Progress of 'reading input':
-- done [took 12.83 s (CPU), 12.94 s (Wall)] -- 
Progress of 'linking features':
-- done [took 7.98 s (CPU), 8.00 s (Wall)] -- 
Number of consensus features:
  of size  6:    208
  of size  5:    811
  of size  4:   1934
  of size  3:   3927
  of size  2:   7589
  of size  1:  23353
  total:       37822
FeatureLinkerUnlabeledQT took 37.58 s (wall), 37.36 s (CPU), 0.00 s (system), 37.36 s (user).


# Post-process data

Extract and prepare the consensus data for normalization - transform it to an appropriate format.

In [21]:
TextExporter \
    -in ${run}/combined.consensusXML \
    -out ${run}/combined.linked_features.csv \
    -consensus:features ${run}/combined.features.csv

Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch2/combined.consensusXML': protein identification run 

Not sure why, but the `combined.features.csv` file included extra empty fields with numbers beyond the actual sample range. These are omitted here, but should probably be remedied upstreams instead.

In [22]:
s_count=$(echo ${samples} | tr " " "\n" | wc -l)
cut_end=$(echo "${s_count} * 5 + 8" | bc)
cut -f 1-${cut_end} ${run}/combined.features.csv \
    > ${run}/combined.features.sub.csv

In [23]:
util_scripts/openms_to_normalyzer.py \
    -i ${run}/combined.features.sub.csv \
    -o ${run}/combined.final.tsv \
    --design ${run}/design.tsv \
    --delim_in '\t' \
    --delim_out '\t'

util_scripts/openms_to_normalyzer.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ms_df = pd.read_csv(args.input, sep=args.delim_in, skiprows=2)
Traceback (most recent call last):
  File "util_scripts/openms_to_normalyzer.py", line 134, in <module>
    main()
  File "util_scripts/openms_to_normalyzer.py", line 23, in main
    normalyzer_df = setup_normalyzer_df(ms_df, design_matrix, require_annot=args.require_annot)
  File "util_scripts/openms_to_normalyzer.py", line 44, in setup_normalyzer_df
    normalyzer_vals = pd.DataFrame(ms_df[target_annot_cols + target_sample_cols].fillna(nan_fill).applymap(str))
  File "/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py", line 2053, in __getitem__
    return self._getitem_array(key)
  File "/usr/local/lib/python3.5/dist-packages/pandas/co

: 1

# Normalization

Perform Loess normalization (maybe even easier in R?)